# 🖼️ Clasificador CNN para CIFAR-10

**Autores:** Alessio Cicilano & Alaeddine Daoudi  
**Fecha:** Octubre 2025

---

## 📋 Descripción

Este notebook implementa una **Red Neuronal Convolucional (CNN)** para clasificar imágenes del dataset CIFAR-10 en 10 categorías:

✈️ Avión | 🚗 Automóvil | 🐦 Pájaro | 🐱 Gato | 🦌 Ciervo | 🐕 Perro | 🐸 Rana | 🐴 Caballo | ⛵ Barco | 🚚 Camión

---

### 🏗️ Arquitectura del Modelo

- **Conv2D** (32 filtros 3x3) + ReLU + MaxPooling
- **Conv2D** (64 filtros 3x3) + ReLU + MaxPooling
- **Flatten**
- **Dense** (64 neuronas) + ReLU
- **Dense** (10 neuronas) + Softmax

**Total parámetros:** ~167.562


## 1️⃣ Configuración e Importación de Librerías


In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU disponible: {tf.config.list_physical_devices('GPU')}")


## 2️⃣ Carga y Preprocesamiento del Dataset


In [ ]:
# Cargar dataset CIFAR-10
print("📥 Cargando dataset CIFAR-10...")
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Nombres de las clases
class_names = ['Avión', 'Automóvil', 'Pájaro', 'Gato', 'Ciervo', 
               'Perro', 'Rana', 'Caballo', 'Barco', 'Camión']

# Información del dataset
print(f"\n📊 Información del Dataset:")
print(f"   - Imágenes de entrenamiento: {train_images.shape[0]:,}")
print(f"   - Imágenes de prueba: {test_images.shape[0]:,}")
print(f"   - Dimensiones de imagen: {train_images.shape[1:]}")
print(f"   - Número de clases: {len(class_names)}")
print(f"   - Rango de píxeles: [{train_images.min()}, {train_images.max()}]")


In [ ]:
# Normalización de las imágenes (0-255 -> 0-1)
print("🔧 Normalizando imágenes...")
train_images_norm = train_images.astype('float32') / 255.0
test_images_norm = test_images.astype('float32') / 255.0

# Convertir etiquetas a one-hot encoding
print("🔧 Codificando etiquetas...")
train_labels_cat = to_categorical(train_labels, 10)
test_labels_cat = to_categorical(test_labels, 10)

print(f"\n✅ Preprocesamiento completado!")
print(f"   - Rango normalizado: [{train_images_norm.min()}, {train_images_norm.max()}]")
print(f"   - Shape etiquetas: {train_labels_cat.shape}")


## 3️⃣ Visualización del Dataset


In [ ]:
# Visualizar 25 imágenes aleatorias del dataset
plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    idx = np.random.randint(0, len(train_images))
    plt.imshow(train_images[idx])
    plt.title(f"{class_names[train_labels[idx][0]]}", fontsize=10, fontweight='bold')
    plt.axis('off')

plt.suptitle('🖼️ Ejemplos del Dataset CIFAR-10', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()


## 4️⃣ Construcción del Modelo CNN


In [ ]:
# Construcción de la arquitectura CNN
print("🏗️ Construyendo modelo CNN...\n")

model = models.Sequential([
    # Primer bloque convolucional
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3), name='conv1'),
    layers.MaxPooling2D((2, 2), name='pool1'),
    
    # Segundo bloque convolucional
    layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
    layers.MaxPooling2D((2, 2), name='pool2'),
    
    # Clasificador
    layers.Flatten(name='flatten'),
    layers.Dense(64, activation='relu', name='dense1'),
    layers.Dense(10, activation='softmax', name='output')
], name='CIFAR10_CNN')

# Resumen del modelo
model.summary()

# Compilación del modelo
print("\n⚙️ Compilando modelo...")
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Modelo compilado correctamente!")


## 5️⃣ Entrenamiento del Modelo


In [ ]:
# Configuración del entrenamiento
EPOCHS = 10
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.1

print(f"🚀 Iniciando entrenamiento...")
print(f"   - Épocas: {EPOCHS}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Validación: {VALIDATION_SPLIT*100}%")
print(f"\nInicio: {datetime.now().strftime('%H:%M:%S')}\n")

# Entrenar el modelo
history = model.fit(
    train_images_norm,
    train_labels_cat,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    verbose=1
)

print(f"\n✅ Entrenamiento completado!")
print(f"Fin: {datetime.now().strftime('%H:%M:%S')}")


## 6️⃣ Evaluación y Visualización de Resultados


In [ ]:
# Evaluar el modelo en el conjunto de prueba
print("🧪 Evaluando modelo en conjunto de prueba...\n")
test_loss, test_accuracy = model.evaluate(test_images_norm, test_labels_cat, verbose=1)

print(f"\n📊 Resultados Finales:")
print(f"   - Pérdida en Test: {test_loss:.4f}")
print(f"   - Precisión en Test: {test_accuracy*100:.2f}%")


In [ ]:
# Gráficos de evolución del entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de Precisión
ax1.plot(history.history['accuracy'], 'b-o', label='Entrenamiento', linewidth=2, markersize=6)
ax1.plot(history.history['val_accuracy'], 'r-s', label='Validación', linewidth=2, markersize=6)
ax1.set_title('📈 Evolución de la Precisión', fontsize=14, fontweight='bold')
ax1.set_xlabel('Época', fontsize=12)
ax1.set_ylabel('Precisión', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Gráfico de Pérdida
ax2.plot(history.history['loss'], 'b-o', label='Entrenamiento', linewidth=2, markersize=6)
ax2.plot(history.history['val_loss'], 'r-s', label='Validación', linewidth=2, markersize=6)
ax2.set_title('📉 Evolución de la Pérdida', fontsize=14, fontweight='bold')
ax2.set_xlabel('Época', fontsize=12)
ax2.set_ylabel('Pérdida', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 7️⃣ Predicciones y Pruebas


In [ ]:
# Predicción detallada de una imagen aleatoria
idx = np.random.randint(0, len(test_images))
image = test_images[idx]
true_label = test_labels[idx][0]

# Predecir
pred_probs = model.predict(test_images_norm[idx:idx+1], verbose=0)[0]
pred_label = np.argmax(pred_probs)

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Imagen
ax1.imshow(image)
ax1.set_title(f"Imagen #{idx}\nClase Real: {class_names[true_label]}", 
             fontsize=12, fontweight='bold')
ax1.axis('off')

# Probabilidades
colors = ['green' if i == pred_label else 'skyblue' for i in range(10)]
bars = ax2.barh(class_names, pred_probs * 100, color=colors)
ax2.set_xlabel('Probabilidad (%)', fontsize=11, fontweight='bold')
ax2.set_title(f'Predicción: {class_names[pred_label]} ({pred_probs[pred_label]*100:.1f}%)',
             fontsize=12, fontweight='bold', 
             color='green' if pred_label == true_label else 'red')
ax2.grid(axis='x', alpha=0.3)

# Añadir valores
for i, (bar, prob) in enumerate(zip(bars, pred_probs * 100)):
    if prob > 2:
        ax2.text(prob + 1, i, f'{prob:.1f}%', va='center', fontsize=9)

plt.tight_layout()
plt.show()

# Resultado
if pred_label == true_label:
    print("✅ ¡Predicción CORRECTA!")
else:
    print(f"❌ Predicción INCORRECTA. Se predijo '{class_names[pred_label]}' pero era '{class_names[true_label]}'")


## 8️⃣ Guardar el Modelo


In [ ]:
# Guardar el modelo entrenado
model_filename = 'cifar10_cnn_model.h5'
model.save(model_filename)
print(f"💾 Modelo guardado como: {model_filename}")

# Tamaño del modelo
import os
if os.path.exists(model_filename):
    model_size = os.path.getsize(model_filename) / (1024 * 1024)
    print(f"📦 Tamaño del modelo: {model_size:.2f} MB")


In [ ]:
# Descargar el modelo (funciona solo en Google Colab)
try:
    from google.colab import files
    files.download(model_filename)
    print(f"⬇️ Descargando modelo...")
except:
    print("ℹ️ No estás en Google Colab. El modelo se ha guardado localmente.")


---

## 📊 Resumen del Proyecto

### 🎯 Objetivos Alcanzados:
- ✅ Carga y preprocesamiento del dataset CIFAR-10
- ✅ Construcción de arquitectura CNN con 2 bloques convolucionales
- ✅ Entrenamiento del modelo con validación
- ✅ Evaluación y análisis de resultados
- ✅ Visualización de predicciones
- ✅ Guardado del modelo entrenado

### 📈 Métricas Esperadas:
- **Precisión en Test**: ~70-75%
- **Total de parámetros**: ~167.562
- **Tiempo de entrenamiento**: ~5-10 minutos (10 épocas con GPU)

### 🚀 Próximos Pasos:
1. Experimentar con más épocas de entrenamiento
2. Añadir capas de Dropout para reducir overfitting
3. Probar Data Augmentation para mejorar generalización
4. Implementar arquitecturas más complejas (ResNet, VGG)

### 🔗 Enlaces Útiles:
- [Repositorio GitHub](https://github.com/Alaedddine718/La_Vision_Artificial)
- [Dataset CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)
- [Documentación TensorFlow](https://www.tensorflow.org/)

---

**Desarrollado por:**  
👨‍💻 Alessio Cicilano  
👨‍💻 Alaeddine Daoudi

**Octubre 2025**

🐍 Python | 🧠 TensorFlow/Keras | 📊 Deep Learning | 🖼️ Computer Vision
